In [2]:
import numpy as np
from astropy.table import Table, vstack
import astropy
import matplotlib.pyplot as plt
import pandas as pd
from astropy.io import ascii
import glob
from jagb_func import custom_hist, simple_gaussian, GLOESS
import statistics
from matplotlib import gridspec
from astropy.coordinates import ICRS, Galactic, Distance, Angle, SkyCoord
from astropy import units as u
from astropy.io import fits
from scipy.optimize import leastsq
from scipy.special import erf
from scipy.optimize import curve_fit
from astropy.wcs import WCS
from scipy import stats


## Define Functions

In [3]:
def compute_rgc(ra,dec, glx_ctr=SkyCoord(10.68458*u.degree, 41.2691*u.degree, frame='icrs'), glx_PA=Angle('38d'), glx_incl=Angle('74d'), glx_dist=Distance(776, unit=u.kpc)):
    """
    Computes deprojected radial distance of a point in a galaxy. All parameters for M31 are from Gregersen+15
    This is modified code based off code by P. Parmby (https://gist.github.com/PBarmby/9355846)

    coord: coordinates of given point
    glx_ctr: galaxy center
    glx_PA: galaxy position angle
    glx_incl: galaxy inclination angle
    glx_dist: distance to galaxy
    """
    coord = SkyCoord(ra*u.degree, dec*u.degree, frame='icrs')
    
    sky_radius = glx_ctr.separation(coord)
    avg_dec = 0.5 * (glx_ctr.dec + coord.dec).radian
    
    x = (glx_ctr.ra - coord.ra) * np.cos(avg_dec)
    y = glx_ctr.dec - coord.dec
    
    phi = glx_PA - Angle('90d') + Angle(np.arctan(y.arcsec / x.arcsec), unit=u.rad)
    xp = (sky_radius * np.cos(phi.radian)).arcmin
    yp = (sky_radius * np.sin(phi.radian)).arcmin

    ypp = yp / np.cos(glx_incl.radian)
    obj_radius = np.sqrt(xp ** 2 + ypp ** 2)  # in arcmin
    obj_dist = Distance(Angle(obj_radius, unit=u.arcmin).radian * glx_dist,
            unit=glx_dist.unit)

    return obj_dist


## Compute the deprojected radius of each field
First, we will compute the average R.A. and Dec. of each field. Then, we can use the function compute_rgc to compute the radial distance to that field from the center of M31. 

In [ ]:
raw_phot_files = sorted(glob.glob('/Users/abigaillee/Photometry/M31 PHAT Photometry/raw PHAT/*')) # these are the fits files from the PHAT team website
order = [21,23,22,18,19,20] # order in which they are being read through glob

average_ra=[]
average_dec=[]
field_nom=[]
brick_nom=[]

for j in range(6): # loop over six bricks
    t =  Table.read(raw_phot_files[j])
    
    # good star cuts
    t=t[(t['f110w_gst']==True)&(t['f814w_gst']==True)&(t['f110w_vega']<90)&(t['f814w_vega']<90)]

    for i in range(18): # loop over 18 fields for each brick
        t_i = t[t['field']==i+1]

        average_ra.append(np.mean(t_i['ra']))
        average_dec.append( np.mean(t_i['dec']))
        field_nom.append(i+1)
        brick_nom.append(order[j])
        
distances = pd.DataFrame({'Central RA':average_ra, 'Central dec':average_dec, 'field_nom':field_nom,'brick_nom':brick_nom})

rgc_list=[]
for i in range(len(distances)):
    rgc_list.append(np.array(compute_rgc(ra=np.array(distances['Central RA'])[i], dec=np.array(distances['Central dec'])[i])))
distances['rgc'] = np.array(rgc_list).flatten()

# sorted distances dataframe now gives the average RGC for each field. 
sorted_distances = distances.sort_values('rgc')

# drop brick 22 field 8 because of earthlimb IR background
sorted_distances=sorted_distances.drop(sorted_distances[(sorted_distances['brick_nom']==22)&(sorted_distances['field_nom']==8)].index)


In [5]:
# show what dataframe looks like
distances = pd.read_csv('/Users/abigaillee/Photometry/M31 PHAT Photometry/raw PHAT/tmp/binned_distances.csv')    
distances.head()

,Unnamed: 0,Central RA,Central dec,field_nom,brick_nom,rgc
0,0,11.655650,42.208827,1,21,16.075355
1,1,11.616019,42.214559,2,21,15.975177
2,2,11.571171,42.219450,3,21,15.969295
3,3,11.526824,42.224715,4,21,16.102721
4,4,11.482915,42.229990,5,21,16.363578
